In [ ]:
#run then restart kernel
!pip install tqdm
!pip install seaborn==0.9.0
!pip install numpy==1.17.3

In [1]:
import sys
print(sys.version)
import warnings
warnings.filterwarnings("ignore")

3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]


In [2]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from time import time
from s3fs.core import S3FileSystem
import psutil
import ipywidgets
from model import *
from sklearn.preprocessing import normalize

In [3]:
start = time()
s3 = S3FileSystem()
embeddings = pickle.load(s3.open('s3://globo-latent/articles_embeddings.pickle'))
df_train = pd.read_csv(s3.open('s3://globo-latent/df_train.csv'))
df_test = pd.read_csv(s3.open('s3://globo-latent/df_test.csv'))
print(time()-start,'seconds to load data.')

7.391472339630127 seconds to load data.


In [4]:
#make them shorter for memory
df_train = df_train[0:5600]
df_test = df_test[df_test.user_id.isin(df_train.user_id.unique())]

## Evaluate Models

### Linear Model
#### Unoptimized

In [17]:
lm = NonlinearModel(
    df_train,
    normalize(embeddings),
    size=8,
    embeddings_size=250,
    T=1,
    norm_U=True
)
lm.evaluate(df_test,threshold=200,max_len=1000)

28.018972158432007 seconds to sort articles by rank.
Mean Average Precision: 0.00018016563458705186
1.2148206233978271 seconds to evaluate.


#### Optimized

In [18]:
start = time()
lm.gradient_descent_nonlinear(
    test=False,
    alpha=0.1,
    max_iterations=300,
    batch_size=16,
    hinge_param = 0,
    readj_interval=1,
    norm_U = False
)
print(time()-start,'seconds to optimize.')
lm.evaluate(df_test,threshold=200,max_len=1000)


405.7609672546387 seconds to optimize.
27.839640140533447 seconds to sort articles by rank.
Mean Average Precision: 0.0003064098076500519
1.2116141319274902 seconds to evaluate.


### Nonlinear Model
#### Unoptimized

In [24]:
nm3 = NonlinearModel(
    df_train,
    normalize(embeddings),
    size=8,
    embeddings_size=250,
    T=3,
    norm_U=True
)
nm3.evaluate(df_test,threshold=200,max_len=1000)

94.74887609481812 seconds to sort articles by rank.
Mean Average Precision: 0.00040639661603632287
1.201690673828125 seconds to evaluate.


#### Optimized

In [25]:
start = time()
nm3.gradient_descent_nonlinear(
    test=False,
    alpha=0.1,
    max_iterations=1000,
    batch_size=32,
    hinge_param = 0.25,
    readj_interval=1,
    norm_U=True
)
print(time()-start,'seconds to optimize.')
nm3.evaluate(df_test,threshold=200,max_len=1000)


3607.254319190979 seconds to optimize.
94.3435697555542 seconds to sort articles by rank.
Mean Average Precision: 0.000513386224411953
1.2039332389831543 seconds to evaluate.
